<a href="https://colab.research.google.com/github/pezkov/gpu_course/blob/main/colab/CudaExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Пример выполнения программы на CUDA**

* Для начала необходимо сменить среду выполнения:

  Перейдите: Среда выполнения → Сменить среду выполнения → Аппаратный ускоритель → Графический процессор Т4

* Для загрузки ваших файлов можно воспользоваться меню слева:

  Файлы → Загрузить в сессонное хранилище

In [ ]:
%%writefile example.cu

#include "cuda_runtime.h"

#include <iostream>

#define CHECK_CUDA_ERROR(call) { 						                            \
    cudaError_t err = call; 							                              \
    if (err != cudaSuccess) { 							                            \
        std::cerr << "Ошибка CUDA (файл: " << __FILE__ << ", строка: "  \
         << __LINE__ << "): " << cudaGetErrorString(err) << std::endl ; \
        return (int32_t)err;							                              \
    } 										                                              \
}


__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

int32_t sum(){
    int a = 2, b = 3, c = 0;
    int *d_a, *d_b, *d_c;

    // Выделяем память на GPU
    CHECK_CUDA_ERROR(cudaMalloc(&d_a, sizeof(int)));
    CHECK_CUDA_ERROR(cudaMalloc(&d_b, sizeof(int)));
    CHECK_CUDA_ERROR(cudaMalloc(&d_c, sizeof(int)));

    // Копируем данные на GPU
    CHECK_CUDA_ERROR(cudaMemcpy(d_a, &a, sizeof(int), cudaMemcpyHostToDevice));
    CHECK_CUDA_ERROR(cudaMemcpy(d_b, &b, sizeof(int), cudaMemcpyHostToDevice));

    // Запускаем ядро
    add<<<1, 1>>>(d_a, d_b, d_c);

    // Проверяем ошибки ядра и синхронизируем
    CHECK_CUDA_ERROR(cudaGetLastError()); // Проверка ошибок запуска ядра
    CHECK_CUDA_ERROR(cudaDeviceSynchronize()); // Синхронизация после ядра

    // Копируем результат обратно на CPU
    CHECK_CUDA_ERROR(cudaMemcpy(&c, d_c, sizeof(int), cudaMemcpyDeviceToHost));

    // Освобождаем память
    CHECK_CUDA_ERROR(cudaFree(d_a));
    CHECK_CUDA_ERROR(cudaFree(d_b));
    CHECK_CUDA_ERROR(cudaFree(d_c));

    std::cout << "Результат: " << a << " + " << b << " = " << c << std::endl;

    return 0;
}

int main() {
    auto err = sum();
    if(err){
      return err;
    }

    return 0;
}

Writing example.cu


Компиляция и запуск кода из примера:

In [ ]:
!nvcc -arch=sm_75 -o example example.cu && ./example

Результат: 2 + 3 = 5


Компиляция и запуск кода из загруженного файла:

https://github.com/pezkov/gpu_course/blob/main/colab/square.cu

In [ ]:
!nvcc -arch=sm_75 -o square square.cu && ./square

launching kernel blocksPerGrid 2, threadsPerBlock 1024
4 9 16 25 36 49 64 81 100 121 144 
